In [261]:
import numpy as np
import time
import threading
import logging
import sys
logger = logging.getLogger(__name__)
# logger.addHandler(logging.StreamHandler(stream=sys.stdout))

def timestamp():
    """Returns the current time as a human readable string."""
    return time.strftime('%y-%m-%d_%Hh%Mm%S', time.localtime())

# class Singleton(type):
#     """
#     Singleton using metaclass.
    
#     Usage:
    
#     class Myclass( MyBaseClass )
#         __metaclass__ = Singleton
    
#     Taken from stackoverflow.com.
#     http://stackoverflow.com/questions/6760685/creating-a-singleton-in-python
#     """
#     _instances = {}
#     def __call__(cls, *args, **kwargs):
#         if cls not in cls._instances:
#             cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
#         return cls._instances[cls]

class StoppableThread( threading.Thread ):
    """
    A thread that can be stopped.
    
    Parameters:
        target:    callable that will be execute by the thread
        name:      string that will be used as a name for the thread
    
    Methods:
        stop():    stop the thread
        
    Use threading.currentThread().stop_request.is_set()
    or threading.currentThread().stop_request.wait([timeout])
    in your target callable to react to a stop request.
    """
    
    def __init__(self, group=None, target=None, name=None):
        super().__init__(group=group, target=target, name=name)
        # super().__init__(self, group, target, name)
        self.stop_request = threading.Event()
        
    def stop(self, timeout=10.):
        name = str(self)
        logger.debug('attempt to stop thread '+name)
        if threading.currentThread() is self:
            logger.debug('Thread '+name+' attempted to stop itself. Ignoring stop request...')
            return
        elif not self.is_alive():
            logger.debug('Thread '+name+' is not running. Continuing...')
            return
        self.stop_request.set()
        self.join(timeout)
        if self.is_alive():
            logger.warning('Thread '+name+' failed to join after '+str(timeout)+' s. Continuing anyway...')

class Singleton(type):
    """
    Singleton using metaclass.
    
    Usage:
    
    class Myclass( MyBaseClass )
        __metaclass__ = Singleton
    
    Modified from stackoverflow.com.
    http://stackoverflow.com/questions/6760685/creating-a-singleton-in-python
    """
    _instances = {}
    def __call__(cls, *args, **kwargs):
        clsname = kwargs["name"] if "name" in kwargs else "default"
        if cls not in cls._instances:
            # no class object is created
            cls._instances[cls] = dict()
            cls._instances[cls][clsname] = super(Singleton, cls).__call__(*args, **kwargs)
        elif clsname not in cls._instances[cls]:
            # some class objects are created but not with the new name
            cls._instances[cls][clsname] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls][clsname]

class JobManager(metaclass=Singleton): 
    """
    Provides a queue for starting and stopping jobs according to their priority.
    
    ToDo: In principle this need not be a singleton. Then there could be different job managers handling different sets of resources. 
          However currently we need singleton since the JobManager is called explicitly on ManagedJob class.    
    """
    def __init__(self):
        self.thread = StoppableThread() # the thread the manager loop is running in
        self.lock = threading.Condition() # lock to control access to 'queue' and 'running'
        self.queue = []
        self.running = None
        self.refresh_interval = 0.1 # seconds
    
    def submit(self, job):

        """
        Submit a job.
        
        If there is no job running, the job is appended to the queue.

        If the job is the running job or the job is already in the queue, do nothing.
        
        If job.priority =< priority of the running job,
            the job is appended to the queue and the queue sorted according to priority.
        
        If job.priority > priority of the running job,
            the job is inserted at the first position of the queue, the running job is stopped
            and inserted again at the first position of the queue.
        """

        logging.debug('Attempt to submit job '+str(job))
        self.lock.acquire()
        
        running = self.running
        queue = self.queue

        if job is running or job in queue:
            logging.info('The job '+str(job)+' is already running or in the queue.')
            self.lock.release()
            return

        queue.append(job)
        # queue.sort(key=lambda job: job.priority, reverse=True) # ToDo: Job sorting not thoroughly tested
        job.state='wait'
                    
        logging.debug('Notifying process thread.')
        self.lock.notify()
            
        self.lock.release()
        logging.debug('Job '+str(job)+' submitted.')
 
    def remove(self, job):
        
        """
        Remove a job.
        
        If the job is running, stop it.
        
        If the job is in the queue, remove it.
        
        If the job is not found, this will result in an exception.
        """
 
        logging.debug('Attempt to remove job '+str(job))
        self.lock.acquire()

        try:
            if job is self.running:
                logging.debug('Job '+str(job)+' is running. Attempt stop.')
                job.stop()
                logging.debug('Job '+str(job)+' removed.')
            else:
                if not job in self.queue:
                    logging.debug('Job '+str(job)+' neither running nor in queue. Returning.')
                else:
                    logging.debug('Job '+str(job)+' is in queue. Attempt remove.')
                    self.queue.remove(job)
                    logging.debug('Job '+str(job)+' removed.')
                    job.state='idle' # ToDo: improve handling of state. Move handling to Job?
        finally:
            self.lock.release()
        
    def start(self):
        """Start the process loop in a thread."""
        if self.thread.is_alive():
            return
        logging.getLogger().info('Starting Job Manager.')
        self.thread = StoppableThread(target = self._process, name=self.__cls_.__name__ + timestamp())
        self.thread.start()
    
    def stop(self, timeout=None):
        """Stop the process loop."""
        self.thread.stop_request.set()
        self.lock.acquire()
        self.lock.notify()
        self.lock.release()        
        self.thread.stop(timeout=timeout)
    
    def _process(self):
        
        """
        The process loop.
        
        Use .start() and .stop() methods to start and stop processing of the queue.
        """
        
        while True:
            
            self.thread.stop_request.wait(self.refresh_interval)
            if self.thread.stop_request.is_set():
                break
            
            # ToDo: jobs can be in queue before process loop is started
            # what happens when manager is stopped while jobs are running?
            
            self.lock.acquire()
            if self.running is None:
                print("run 1")
                if self.queue == []:
                    logging.debug('No job running. No job in queue. Waiting for notification.')
                    self.lock.wait()
                    logging.debug('Caught notification.')
                    if self.thread.stop_request.is_set():
                        self.lock.release()        
                        break
                logging.debug('Attempt to fetch first job in queue.')
                self.running = self.queue.pop(0)
                logging.debug('Found job '+str(self.running)+'. Starting.')
                self.running.start()
            elif not self.running.thread.is_alive():
                print("run 2")
                logging.debug('Job '+str(self.running)+' stopped.')
                self.running=None
                if self.queue != []:
                    logging.debug('Attempt to fetch first job in queue.')
                    self.running = self.queue.pop(0)
                    logging.debug('Found job '+str(self.running)+'. Starting.')
                    self.running.start()
            elif self.queue != [] and self.queue[0].priority > self.running.priority:
                print("run 3")
                logging.debug('Found job '+str(self.queue[0])+' in queue with higher priority than running job. Attempt to stop running job.')            
                self.running.stop()
                if self.running.state != 'done':
                    logging.debug('Reinserting job '+str(self.running)+' in queue.')
                    self.queue.insert(0,self.running)
                    self.queue.sort(key=lambda job: job.priority, reverse=True) # ToDo: Job sorting not thoroughly tested
                    self.running.state='wait'
                self.running = self.queue.pop(0)
                logging.debug('Found job '+str(self.running)+'. Starting.')
                self.running.start()                
            self.lock.release() 


In [262]:
from nspyre import InstrumentGateway
from nspyre import DataSource
from rpyc.utils.classic import obtain


class DummyJob(metaclass=Singleton):
    _refresh_interval = 0.001
    _to_keep = False  # whether to keep the data when the thread is stopped (idx_run<=num_run)

    name = "Dummy_task"

    priority = 1 # from 1 to 10
    state = "idle" # 'idle', 'run', 'wait', 'done', 'error'
    thread = StoppableThread()

    time_mea = 0.0 # readonly, accumulated measurement time
    num_run = 100 # number of repetition, readonly
    idx_run = 0 # indicating which iteration we are at, 1-based
    

    # buffer = np.array([], dtype=np.float64, order='C')
    # buffer should be handled in the hardware class object
    paraset = dict()
    dataset = dict(raw=[[]]) # store the raw data of each iteration

    def __init__(self, 
        name="Dummy_task"
        ):
        self.name = name
        self.gw = InstrumentGateway()
        self.ds = DataSource(self.name)


    def set_expname(self, name):
        self.name = name

    def set_priority(self, order):
        self.priority = order

    def set_runnum(self, num):
        self.num_run = num

    def set_keepdata(self, to_keep):
        # to_keep (bool)
        self.keep_data = to_keep

    def set_paraset(self, **para_dict):
            # set parametes
        for kk, vv in para_dict.items():
            self.paraset[kk] = vv

    # methods for handling the run thread====================================
    def start(self):
        self.thread = StoppableThread(target = self.__run, name=self.__cls_.__name__ + str(time.time()))
        self.thread.start()

    def pause(self, timeout=None):
        """Stop the process loop."""
        self.thread.stop_request.set()  
        self.thread.stop(timeout=timeout)
        self._to_keep = True
    
    def stop(self, timeout):
        self.pause(timeout)
        self._to_keep = False # this must be placed after pause()
    # ======================================================================


    def _setup_exp(self):
        # check the parameters if needed -------------------------------------
        if not self.keep_data:
            self.idx_run = 0
            self.run_time = 0
            self.state = "idle"
            # reset the dataset
            self.dataset = dict()
        
        self.ds.start()
        # --------------------------------------------------------------------
        self.gw.connect()
        # setup the hardwares here--------------------------------------------
        # # gw.aninstrument.set_something(self.paraset["var1"])
        # --------------------------------------------------------------------


    def _run_exp(self):
        # run the experiment
        self.gw.nidaq.read_data()
        
    def _organize_data(self):
        stateset = dict(
            priority=self.priority,
            state=self.state,
            run_time=self.run_time,
            idx_run=self.idx_run, 
            num_run=self.num_run
        )
        
        to_dataserv = dict(
            # name=__name__,
            stateset= stateset,
            paraset = self.paraset, 
            dataset = self.dataset
        )
        # # push the data to the data server
        # with self.ds as pipe:
        #     pipe.push(to_dataserv)

    def _handle_exp_error(self):
        pass

    def _shutdown_exp(self):

        self.ds.stop()

        # set the hardwares here ------------------------------------
        # # gw.aninstrument.set_something(self.paraset["var1"])
        self.gw.disconnect()
        # -----------------------------------------------------------
        

    def __run(self):
        """
        Method that is running in a thread.
        It should NOT be modified or called in any classes of the lower hierachy
        """
        try:
            self.state='run'
            start_time = time.time()
            self.run_time = 0
            self._setup_exp()
            for _ in range(self.num_run):
                self.thread.stop_request.wait(self._refresh_interval)
                if self.thread.stop_request.is_set() or self.idx_run==self.num_run:
                    logger.debug('Received stop signal. Returning from thread.')
                    break
                
                self._run_exp()
                
                curr_time = time.time()
                self.run_time = curr_time - start_time
                print(f"time time:{self.run_time}")
                self.idx_run += 1
                self._organize_data()
            else:
                if self.num_run == 0:
                    self.state = "idle"
                else:
                    self.state='done'
        except:
            logger.exception('Error in job.')
            self.state='error'
            self._handle_exp_error()
        finally:
            logger.debug('Reseting all instruments.')  
            self._shutdown_exp()
        


In [263]:
dumjob = DummyJob(name="fdsfs")

In [268]:
dumjob.ggg = 6

In [269]:
dumjob2 = DummyJob(name="fdsfs")

In [271]:
dumjob

In [272]:
dumjob2.ggg = 4532

In [273]:
dumjob.ggg

4532

In [67]:
class Namedclass(object):
    _instance = None
    def __new__(cls, *args, **kwargs):
        if not isinstance(cls._instance, cls):
            cls._instance = object.__new__(cls, *args, **kwargs)
        return cls._instance

    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

In [259]:
class Sing(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        print(args)
        print(kwargs)
        print(cls._instances)
        clsname = kwargs["name"] if "name" in kwargs else "default"
        if cls not in cls._instances:
            # no class object is created
            cls._instances[cls] = dict()
            cls._instances[cls][clsname] = super(Sing, cls).__call__(*args, **kwargs)
        elif clsname not in cls._instances[cls]:
            # some class objects are created but not with the new name
            cls._instances[cls][clsname] = super(Sing, cls).__call__(*args, **kwargs)
        print(cls._instances)
        return cls._instances[cls][clsname]
class shitclass(metaclass=Sing):
    def __init__(self, var1):
        self.var1 = var1

class dumclass(metaclass=Sing):
    def __init__(self, name=""):
        self._name = name

class dumshitclass(metaclass=Sing):
    def __init__(self, var1, name=""):
        self.var1 = var1
        self._name = name
class wrapdumclass(dumclass):
    def __init__(self, name, *args, **kwargs):
        super().__init__(*args, name=name, **kwargs)

In [251]:
sss = shitclass(46543)

(46543,)
{}
{}
{<class '__main__.shitclass'>: {'default': <__main__.shitclass object at 0x0000018191460790>}}


In [256]:
dddd = dumclass("jjjfdsf")

('jjjfdsf',)
{}
{<class '__main__.shitclass'>: {'default': <__main__.shitclass object at 0x0000018191460790>}, <class '__main__.dumclass'>: {'fdsf': <__main__.dumclass object at 0x00000181914618D0>}, <class '__main__.dumshitclass'>: {'fdsf': <__main__.dumshitclass object at 0x00000181914856F0>, 'hibye': <__main__.dumshitclass object at 0x00000181905B3FD0>}}
{<class '__main__.shitclass'>: {'default': <__main__.shitclass object at 0x0000018191460790>}, <class '__main__.dumclass'>: {'fdsf': <__main__.dumclass object at 0x00000181914618D0>, 'default': <__main__.dumclass object at 0x0000018191485900>}, <class '__main__.dumshitclass'>: {'fdsf': <__main__.dumshitclass object at 0x00000181914856F0>, 'hibye': <__main__.dumshitclass object at 0x00000181905B3FD0>}}


In [254]:
dsds = dumshitclass(54353, name="fdsf")

(54353,)
{'name': 'fdsf'}
{<class '__main__.shitclass'>: {'default': <__main__.shitclass object at 0x0000018191460790>}, <class '__main__.dumclass'>: {'fdsf': <__main__.dumclass object at 0x00000181914618D0>}, <class '__main__.dumshitclass'>: {}}
{<class '__main__.shitclass'>: {'default': <__main__.shitclass object at 0x0000018191460790>}, <class '__main__.dumclass'>: {'fdsf': <__main__.dumclass object at 0x00000181914618D0>}, <class '__main__.dumshitclass'>: {'fdsf': <__main__.dumshitclass object at 0x00000181914856F0>}}


In [255]:
dsddd = dumshitclass(54353, name="hibye")

(54353,)
{'name': 'hibye'}
{<class '__main__.shitclass'>: {'default': <__main__.shitclass object at 0x0000018191460790>}, <class '__main__.dumclass'>: {'fdsf': <__main__.dumclass object at 0x00000181914618D0>}, <class '__main__.dumshitclass'>: {'fdsf': <__main__.dumshitclass object at 0x00000181914856F0>}}
{<class '__main__.shitclass'>: {'default': <__main__.shitclass object at 0x0000018191460790>}, <class '__main__.dumclass'>: {'fdsf': <__main__.dumclass object at 0x00000181914618D0>}, <class '__main__.dumshitclass'>: {'fdsf': <__main__.dumshitclass object at 0x00000181914856F0>, 'hibye': <__main__.dumshitclass object at 0x00000181905B3FD0>}}


In [260]:
wds = wrapdumclass("yo")

('yo',)
{}
{}
{<class '__main__.wrapdumclass'>: {'default': <__main__.wrapdumclass object at 0x0000018191562470>}}


In [230]:
class MultiSing(object):
    """ A silly doc"""
    _instance = {}
    apple = 555
    orange = "pen"
    banana = "one"
    def __new__(cls, name, *args, **kwargs):
        # clsname = cls.__class__.__name__
        clsname = str(cls)+name
        # print(cls)
        # print(clsname)
        # print(cls.__dict__)
        # print(str(cls)+"sdfds")
        print(cls._instance)
        if cls._instance == {}:
            print("make the first class object")
            cls._instance[name] = object.__new__(cls, *args, **kwargs)
            # cls._instance[clsname] = object.__new__(cls)
        elif clsname not in cls._instance:
            print("not the same class object")
            cls._instance[name] = object.__new__(cls, *args, **kwargs)
            # cls._instance[clsname] = object.__new__(cls)
            print("name unmatched, different class object")
        else:
            print("name matched, same class object")
        print(cls._instance)
        return cls._instance[clsname]
        
    def __init__(self, name, *args, **kwargs):
        # super().__init__(*args, **kwargs)
        self._name = name
        self.argsss = args
        self.kwargsss = kwargs

class SubMS(MultiSing):

    def __init__(self, name, epicarg, akwa = 544):
        super().__init__( name, epicarg, akwa = 544)
        self.epicarg = epicarg
        self.akwa = akwa

In [231]:
sub1 = SubMS("qqq", 6565, akwa=32432)

{}
make the first class object


TypeError: object.__new__() takes exactly one argument (the type to instantiate)

In [225]:
sub2 = SubMS("qqq", 343)

{"<class '__main__.SubMS'>qqq": <__main__.SubMS object at 0x000001819139C2B0>}
name matched, same class object
{"<class '__main__.SubMS'>qqq": <__main__.SubMS object at 0x000001819139C2B0>}
343


In [226]:
print(sub1.epicarg, sub1.akwa)

343 544


In [189]:
sub2 = SubMS("abc")

{'abcddssf': <__main__.MultiSing object at 0x000001819062ED10>, 'abc': <__main__.MultiSing object at 0x00000181912B9240>, 'ttt': <__main__.SubMS object at 0x00000181912B9FF0>}
name matched, same class object
{'abcddssf': <__main__.MultiSing object at 0x000001819062ED10>, 'abc': <__main__.MultiSing object at 0x00000181912B9240>, 'ttt': <__main__.SubMS object at 0x00000181912B9FF0>}


In [190]:
sub1.dsfsdf

AttributeError: 'SubMS' object has no attribute 'dsfsdf'

In [165]:
qqwwee = MultiSing("abcddssf")

{}
make the first class object
{'abcddssf': <__main__.MultiSing object at 0x000001819062ED10>}


In [199]:
qqwwee.__dict__

{'_name': 'abcddssf', 'argsss': (6666,), 'kwargsss': {}, 'banana': 'two'}

In [200]:
qqwwee.__class__.__name__

'MultiSing'

In [166]:
print(qqwwee.argsss)

()


In [167]:
dfsf = MultiSing("abcddssf", 6666)

{'abcddssf': <__main__.MultiSing object at 0x000001819062ED10>}
name matched, same class object
{'abcddssf': <__main__.MultiSing object at 0x000001819062ED10>}


In [168]:
fsdfds = MultiSing("abc")

{'abcddssf': <__main__.MultiSing object at 0x000001819062ED10>}
not the same class object
name unmatched, different class object
{'abcddssf': <__main__.MultiSing object at 0x000001819062ED10>, 'abc': <__main__.MultiSing object at 0x00000181912B9240>}


In [172]:
qqwwee._instance

{'abcddssf': <__main__.MultiSing at 0x1819062ed10>,
 'abc': <__main__.MultiSing at 0x181912b9240>}

In [176]:
qqwwee.banana = "two"

In [171]:
vdsvs = MultiSing("abc")

{'abcddssf': <__main__.MultiSing object at 0x000001819062ED10>, 'abc': <__main__.MultiSing object at 0x00000181912B9240>}
name matched, same class object
{'abcddssf': <__main__.MultiSing object at 0x000001819062ED10>, 'abc': <__main__.MultiSing object at 0x00000181912B9240>}


In [179]:
vdsvs.banana

'ttheh'

In [178]:
fsdfds.banana = "ttheh"

In [143]:
fsdfds.pp = "500"

In [146]:
print(qqwwee.pp)

AttributeError: 'NoneType' object has no attribute 'pp'

In [101]:
qwe = shitshit2("hi")

In [102]:
fsfds = shitshit2("hi")

name matched


In [103]:
dsfds = shitshit2("dsfds")

In [87]:
shihis = shitclass("fsdf")
print(shihis)
print(type(shihis))

<class '__main__.shitclass'>
{}
{<class '__main__.shitclass'>: <__main__.shitclass object at 0x0000018190540B20>}


In [94]:
shihis.__dict__

{'_name': 'fsdf'}

In [88]:
abcc = shitclass("fd453")
print(abcc)
print(type(abcc))

<class '__main__.shitclass'>


In [89]:
shihis._name

'fsdf'

In [90]:
abcc._name

'fsdf'

In [72]:
apple = Namedclass()(2,3,4)

TypeError: Namedclass.__init__() missing 3 required positional arguments: 'a', 'b', and 'c'

In [63]:
def set_paraset(**para_dict):
        # set parametes
    paraset = dict()
    for kk, vv in para_dict.items():
        paraset[kk] = vv
    print(paraset)

In [64]:
set_paraset(abc=1, apple=2345)

{'abc': 1, 'apple': 2345}


In [55]:
jobmanager = JobManager()
jobmanager.start()
time.sleep(0.1)



#import time
#time.sleep(0.1)

#j3 = Job()
#j3.priority = 1
#JobManager().submit(j3)
# JobManager().stop()
    

run 1


In [56]:
jobs = [ DummyJob() for i in range(1)]
    
jobs[0].priority = 4
jobs[0].num_run = 100
jobs[0].refresh_interval = 0.000001
# jobs[1].priority = 1
# jobs[2].priority = 0
# jobs[3].priority = 10
# jobs[4].priority = 0

for job in jobs:
    JobManager().submit(job) 

time.sleep(0.1)
q = JobManager().queue

print([job.priority for job in q])
print([q.index(job) if job in q else None for job in jobs])

time.sleep(2)
JobManager().remove(job)


time time:0.0050084590911865234
ran experiment once
time time:0.020243406295776367
ran experiment once
time time:0.03618359565734863
ran experiment once
time time:0.05118417739868164
ran experiment once
time time:0.0671842098236084
ran experiment once
time time:0.08218550682067871
ran experiment once
time time:0.09818410873413086
ran experiment once
time time:0.11331844329833984
ran experiment once
[]
[None]
time time:0.1293187141418457
ran experiment once
time time:0.1453249454498291
ran experiment once
time time:0.16122794151306152
ran experiment once
time time:0.17600250244140625
ran experiment once
time time:0.19172096252441406
ran experiment once
time time:0.20676279067993164
ran experiment once
time time:0.22252345085144043
ran experiment once
time time:0.238525390625
ran experiment once
time time:0.2545621395111084
ran experiment once
time time:0.27057933807373047
ran experiment once
time time:0.2865898609161377
ran experiment once
time time:0.3025786876678467
ran experiment onc

In [21]:
JobManager().submit(DummyJob()) 

ran experiment once
ran experiment once
ran experiment once
ran experiment once
ran experiment once
ran experiment once


In [17]:
JobManager().running

In [18]:
JobManager().stop()

C:\Users\ctcheung\AppData\Local\Temp\ipykernel_24756\1730625352.py:54: DeprecationWarning: currentThread() is deprecated, use current_thread() instead
  if threading.currentThread() is self:


In [7]:
print([job.priority for job in q])
print([q.index(job) if job in q else None for job in jobs])


[1]
[None]


In [8]:
print(q)

In [1]:
from nspyre import DataSource

In [2]:
ds = DataSource("fdsfdsfdsafdsafas")

In [3]:
ds.start()


In [6]:
ds.push({"fdsfds":[4,5,56,2,2]})

In [8]:
ddss = DataSource("iiiii")

In [9]:
ddss.start()

In [10]:
ddss.push(["fdsfdsfdsfds"])